<a href="https://colab.research.google.com/github/arssite/AMPLOYMENT---for-Employers-and-Employees-/blob/main/ATS%20RESUME%20OPTIMIZER/ATSRESUME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch gradio python-docx spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/1

In [ ]:
import gradio as gr
from transformers import pipeline
import spacy
from docx import Document
import tempfile

In [ ]:
nlp = spacy.load('en_core_web_sm')
fill_mask = pipeline("fill-mask", model="bert-base-uncased", device=0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def extract_text_from_docx(file_path):
    try:
        doc = Document(file_path)
        return '\n'.join([para.text for para in doc.paragraphs])
    except:
        return ""


In [ ]:
def save_updated_resume(text):
    try:
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.docx')
        doc = Document()
        for line in text.split('\n'):
            doc.add_paragraph(line)
        doc.save(temp_file.name)
        return temp_file.name
    except:
        return None

In [ ]:
def bert_suggest_replacements(text):
    suggestions = []
    improved_text = text
    doc = nlp(text)

    for token in doc:
        if token.is_stop or not token.is_alpha or len(token.text) <= 3:
            continue

        masked_sentence = improved_text.replace(token.text, "[MASK]", 1)
        try:
            predictions = fill_mask(masked_sentence)
            best_suggestion = predictions[0]['token_str']

            if best_suggestion != token.text.lower():
                improved_text = improved_text.replace(token.text, best_suggestion, 1)
                suggestions.append(f"• Replace '{token.text}' with '{best_suggestion}'")
        except:
            continue

    return improved_text, suggestions

In [ ]:
def process_resume(file):
    if file is None:
        return "Please upload a resume file.", "No suggestions yet.", None

    try:
        resume_text = extract_text_from_docx(file.name)
        if not resume_text:
            return "Could not read the resume. Please ensure it's a valid .docx file.", "No suggestions available.", None

        updated_text, suggestions = bert_suggest_replacements(resume_text)
        output_file_path = save_updated_resume(updated_text)

        suggestions_text = "Suggested Improvements:\n" + '\n'.join(suggestions) if suggestions else "No improvements suggested. Your resume looks good!"

        return updated_text, suggestions_text, output_file_path
    except:
        return "An error occurred. Please try again with a different file.", "No suggestions available.", None


In [ ]:
iface = gr.Interface(
    fn=process_resume,
    inputs=gr.File(label="Upload Your Resume (.docx only)", file_types=[".docx"]),
    outputs=[
        gr.Textbox(label="Optimized Resume Text", lines=10),
        gr.Textbox(label="ATS Improvement Suggestions", lines=10),
        gr.File(label="Download Optimized Resume")
    ],
    title="ATS Resume Optimizer",
    description="Upload your .docx resume for ATS optimization. We'll suggest improvements and provide an updated version.",
    theme="default",
    css="""
        .gradio-container {max-width: 800px; margin: auto;}
        .output-markdown {white-space: pre-wrap;}
    """
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7fd00a7347a8aec2b8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [13]:
!gradio deploy

Need 'write' access token to create a Spaces repo.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git cre